In [1]:
from pathlib import Path
from torch.utils.data import DataLoader
import sys
sys.path.append('../')
from scLinguist.data_loaders.data_loader import scMultiDataset
from scLinguist.model.configuration_hyena import HyenaConfig
from scLinguist.model.model import scTrans
import importlib, sys
sys.modules['model'] = importlib.import_module('scLinguist.model')

ENCODER_CKPT = Path("../pretrained_model/encoder.ckpt")
DECODER_CKPT = Path("../pretrained_model/decoder.ckpt")
FINETUNE_CKPT = Path("../pretrained_model/finetune.ckpt")
SAVE_DIR = Path("../docs/tutorials/zeroshot_output")
SAVE_DIR.mkdir(exist_ok=True)

In [2]:
enc_cfg = HyenaConfig(
    d_model        = 128,
    emb_dim        = 5,
    max_seq_len    = 19202,
    vocab_len      = 19202,
    n_layer        = 1,
    output_hidden_states=False,
)
dec_cfg = HyenaConfig(
    d_model        = 128,
    emb_dim        = 5,
    max_seq_len    = 6427,
    vocab_len      = 6427,
    n_layer        = 1,
    output_hidden_states=False,
)
model = scTrans.load_from_checkpoint(checkpoint_path=FINETUNE_CKPT)
model.encoder_ckpt_path = ENCODER_CKPT
model.decoder_ckpt_path = DECODER_CKPT
model.mode = "RNA-protein"

In [3]:
import scanpy as sc
import torch

zeroshot_adata = sc.read_h5ad("../data/test_sample_rna.h5ad")[:10]
zeroshot_rna_tensor = torch.tensor(zeroshot_adata.X.todense(), dtype=torch.float32).cuda()

In [4]:
model.eval().cuda()

with torch.no_grad():
    _, _, protein_pred = model(zeroshot_rna_tensor)

In [5]:
target_proteins = [line.strip() for line in open("../docs/tutorials/protein_names.txt")]

import pandas as pd
prot_map = pd.read_csv("../docs/tutorials/protein_index_map.csv")
name_to_idx = dict(zip(prot_map["name"], prot_map["index"]))

idx = [name_to_idx[p] for p in target_proteins if p in name_to_idx]

pred_df = pd.DataFrame(
    protein_pred[:, idx].cpu().numpy(),
    columns = target_proteins,
    index   = zeroshot_adata.obs_names,
)
pred_df.to_csv(SAVE_DIR/"predicted_protein_expression.csv")